This script wil take the initial fire scenes downloaded from earth engine and split them into 256x256 chunks as long as there is a fire present in the chunk.
It will also create the training, testing and validation datasets and files

import packages

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras
import random
import numpy as np
import xarray as xr
import rioxarray
#import seaborn as sns
from sklearn.utils import shuffle
from skimage.util.shape import view_as_blocks
import glob
from MightyMosaic import MightyMosaic

2023-07-09 08:47:39.712614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 08:47:39.818756: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Break images into 256 by 256 and remove any where there is no target variable (no 1's).

In [2]:
"""arguments are path to .tif files to be chunked, out_path to save files, chunk size, and if desired a threshold"""
in_path = '/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/nbac_modis'
out_path = '/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/nbac_modis'


size = 128
threshold = True

if threshold == True:

    out_path = out_path + '_subs_0_' + str(size)
    if not os.path.isdir(out_path):
        os.makedirs(out_path)

if threshold == False:

    out_path = out_path + '_subs_' + str(size)
    if not os.path.isdir(out_path):
        os.makedirs(out_path)

files = os.listdir(in_path)

for f in files:

    if f.endswith('.tif'):

        f2 = f.replace('modis_', '')

        #in file
        try:
            in_mat = rioxarray.open_rasterio(os.path.join(in_path, f)).to_numpy()

            #convert to band last
            in_mat = np.moveaxis(in_mat, 0, 2) 
            
            # frac = int(0.22 * in_mat.shape[0])
            # test = test[:, :, -3]
            # in_mat = in_mat[frac:-frac, frac:-frac, :]



            mosaic = MightyMosaic.from_array(in_mat, (size,size), overlap_factor=1) 


            #first two dimensions are number of chunks, 3 and 4 are size (256, 266) and last is nbands
            for i in range(mosaic.shape[0]):
                
                for j in range(mosaic.shape[1]):

                    fname = os.path.join(str(i) + '_' +  str(j)+ '_' + f2.replace('.tif', '.npy'))
                    out_name = os.path.join(out_path, fname)

                    if os.path.exists(out_name) == False:

                        in_mat3 = mosaic[i, j, :, :, :-1]

                        target = mosaic[i, j, :, :, 10]
                        

                        #turn nan to -999
                        in_mat[np.isnan(in_mat)] = -999

                        target = target.astype('int')

                        target[target <0 ] = 0
                        target[target >1 ] = 0

                        #
                        target[~np.isin(target, [0,1])] = 0

                        #turn nan to 0
                        target[np.isnan(target)] = 0
                        
                        # if np.all(target == 0) == False:

                        #now get dnbr which is the 6th band
                        dnbr = in_mat3[:, :, 7]

        #                         #if threshold apply dnbr threshold
        #                         if theshold:
                        target[dnbr < 0] = 0

                        e = np.dstack([in_mat3, target])

                        np.save(out_name, e)

    #                         else:

    #                 e = np.dstack([in_mat3, target])

    #                 np.save(out_name, e)
                        print(f)

        except:
            pass

modis_10000.tif
modis_10001.tif
modis_10002.tif
modis_10003.tif
modis_10004.tif
modis_10005.tif
modis_10006.tif
modis_10007.tif
modis_10008.tif
modis_10009.tif
modis_10010.tif
modis_10011.tif
modis_10013.tif
modis_10014.tif
modis_10012.tif
modis_10015.tif
modis_10016.tif
modis_10017.tif
modis_10018.tif
modis_10019.tif
modis_10020.tif
modis_10021.tif
modis_10022.tif
modis_10023.tif
modis_10024.tif
modis_10025.tif
modis_10027.tif
modis_10026.tif
modis_10029.tif
modis_10028.tif
modis_10030.tif
modis_10033.tif
modis_10031.tif
modis_10032.tif
modis_10038.tif
modis_10034.tif
modis_10035.tif
modis_10039.tif
modis_10037.tif
modis_10036.tif
modis_10041.tif
modis_10042.tif
modis_10040.tif
modis_10044.tif
modis_10043.tif
modis_10058.tif
modis_10050.tif
modis_10056.tif
modis_10061.tif
modis_10045.tif
modis_10046.tif
modis_10047.tif
modis_10067.tif
modis_10051.tif
modis_10052.tif
modis_10055.tif
modis_10054.tif
modis_10053.tif
modis_10071.tif
modis_10057.tif
modis_10073.tif
modis_10072.tif
modis_10

In [3]:
't'

't'

In [5]:

def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]
#list of files
# file_names = listdir_fullpath('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/nbac_1985_sent_harm_subs_128')
file_names = listdir_fullpath('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/nbac_modis_subs_0_128')


#shuffle file_names
file_names = shuffle(file_names, random_state = 555)


print(len(file_names))

random.seed(555)
# #get train at 80%

train_files = random.sample(file_names, k=round(len(file_names) * 0.8))

#files at 10%
val_files = list(set(file_names) - set(train_files))
val_files = random.sample(val_files, k=round(len(file_names) * 0.1))

#combine traini and val
temp = train_files + val_files
#get test files at 10%
test_files = list(set(file_names) - set(temp))


#convert the lists to pandas dataframes
train = pd.DataFrame({'Files': train_files})
val = pd.DataFrame({'Files': val_files})
test = pd.DataFrame({'Files': test_files})

print(train.shape)
print(val.shape)
print(test.shape)

train.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/nbac_modis_0_128_training_files.csv')
val.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/nbac_modis_0_128_validation_files.csv')
test.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/nbac_modis_0_128_testing_files.csv')

8792
(7034, 1)
(879, 1)
(879, 1)
